<a href="https://colab.research.google.com/github/dakilaledesma/VIGOR2/blob/main/notebooks/%5BNI%5D_Kinematics_to_Music_Protomodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
! curl -L https://herbarium.ucht.cloud/s/AoZFbaKWofEfHiF/download --output motions.zip
! curl -L https://herbarium.ucht.cloud/s/Cn55KNbEBqtrDCg/download --output mp3.zip
! curl -L https://herbarium.ucht.cloud/s/MayD6rAGWoKmGw8/download --output keypoints3d.zip
! unzip -q motions.zip
! unzip -q keypoints3d.zip
! unzip -q mp3.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  305M  100  305M    0     0  30.0M      0  0:00:10  0:00:10 --:--:-- 46.3M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 90.6M  100 90.6M    0     0  25.1M      0  0:00:03  0:00:03 --:--:-- 25.1M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  834M  100  834M    0     0  45.6M      0  0:00:18  0:00:18 --:--:-- 48.3M


In [14]:
audio_dict = {}
motion_dict = {}

In [15]:
# Thanks to https://stackoverflow.com/questions/69387104/how-to-convert-wav-audio-file-from-mel-spectrogram
import librosa
import os
from glob import glob

for aud in glob("mp3/*.mp3"):
  my_audio_as_np_array, my_sample_rate = librosa.load(aud)

  spec = librosa.feature.melspectrogram(y=my_audio_as_np_array,
                                          sr=my_sample_rate, 
                                              n_fft=2048, 
                                              hop_length=512, 
                                              win_length=None, 
                                              window='hann', 
                                              center=True, 
                                              pad_mode='reflect', 
                                              power=2.0,
                                      n_mels=128)
  spec = spec.T[:1272].T # Minimum is (128, 1273)
  bn = os.path.basename(aud).replace(".mp3", '')
  audio_dict[bn] = spec

In [16]:
import pickle
import numpy as np
import matplotlib.pyplot as plt

for fn in glob("keypoints3d/*.pkl"):
  with open(fn, "rb") as motion:
    m = pickle.load(motion)
    bn = os.path.basename(fn).replace(".pkl", '')
    motion_dict[bn] = m["keypoints3d"][:425] # Minimum is (426, 17, 3), axis=0 is all different

In [17]:
print(motion_dict.keys())

dict_keys(['gLH_sBM_cAll_d18_mLH5_ch06', 'gBR_sBM_cAll_d04_mBR0_ch03', 'gKR_sFM_cAll_d29_mKR5_ch13', 'gMH_sBM_cAll_d24_mMH5_ch02', 'gWA_sBM_cAll_d26_mWA5_ch04', 'gJB_sFM_cAll_d08_mJB0_ch08', 'gLO_sBM_cAll_d13_mLO1_ch03', 'gKR_sBM_cAll_d28_mKR3_ch09', 'gHO_sBM_cAll_d19_mHO3_ch02', 'gJS_sBM_cAll_d03_mJS5_ch08', 'gKR_sBM_cAll_d29_mKR0_ch07', 'gKR_sBM_cAll_d30_mKR4_ch10', 'gMH_sBM_cAll_d24_mMH5_ch07', 'gKR_sBM_cAll_d30_mKR3_ch10', 'gLH_sBM_cAll_d18_mLH4_ch05', 'gBR_sBM_cAll_d05_mBR1_ch02', 'gBR_sBM_cAll_d06_mBR4_ch09', 'gLH_sBM_cAll_d18_mLH4_ch01', 'gWA_sBM_cAll_d26_mWA0_ch06', 'gHO_sBM_cAll_d20_mHO5_ch10', 'gKR_sFM_cAll_d28_mKR3_ch07', 'gLH_sBM_cAll_d17_mLH5_ch06', 'gKR_sBM_cAll_d30_mKR3_ch06', 'gMH_sBM_cAll_d22_mMH0_ch08', 'gLH_sFM_cAll_d16_mLH2_ch03', 'gWA_sFM_cAll_d26_mWA2_ch10', 'gLH_sBM_cAll_d18_mLH5_ch10', 'gMH_sBM_cAll_d22_mMH1_ch03', 'gKR_sBM_cAll_d29_mKR0_ch04', 'gLO_sBM_cAll_d14_mLO4_ch03', 'gKR_sBM_cAll_d29_mKR5_ch03', 'gLH_sFM_cAll_d16_mLH0_ch01', 'gLO_sBM_cAll_d15_mLO2_ch04',

In [18]:
print(audio_dict.keys())

dict_keys(['mJB1', 'mPO4', 'mWA2', 'mWA1', 'mBR4', 'mJB4', 'mLO1', 'mWA0', 'mLH0', 'mBR0', 'mHO3', 'mLH1', 'mMH0', 'mKR0', 'mHO2', 'mLO3', 'mMH5', 'mKR3', 'mBR1', 'mKR1', 'mKR2', 'mJS4', 'mLO4', 'mLH5', 'mJB3', 'mJB0', 'mLO2', 'mKR5', 'mJS1', 'mWA4', 'mPO1', 'mLH2', 'mBR3', 'mMH1', 'mPO2', 'mMH3', 'mLH4', 'mLH3', 'mMH4', 'mBR5', 'mPO0', 'mLO0', 'mHO0', 'mJB5', 'mHO4', 'mBR2', 'mKR4', 'mHO5', 'mJB2', 'mPO3', 'mJS5', 'mJS0', 'mLO5', 'mWA5', 'mJS2', 'mPO5', 'mHO1', 'mWA3', 'mMH2', 'mJS3'])


In [20]:
X = []
y = []
for ak in audio_dict.keys():
  mks = [k for k in motion_dict.keys() if ak in k]
  for mk in mks:
    X.append(motion_dict[mk])
    y.append(audio_dict[ak])

In [22]:
X = np.array(X)
y = np.array(y)
print(X.shape, y.shape)

(1408, 425, 17, 3) (1408, 128, 1272)


In [ ]:
import tensorflow.keras as keras
from tensorflow.keras import layers

input_img = keras.Input(shape=(426, 17, 3))

x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

# at this point the representation is (4, 4, 8) i.e. 128-dimensional

x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(16, (3, 3), activation='relu')(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = keras.Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
res = librosa.feature.inverse.mel_to_audio(spec, 
                                           sr=my_sample_rate, 
                                           n_fft=2048, 
                                           hop_length=512, 
                                           win_length=None, 
                                           window='hann', 
                                           center=True, 
                                           pad_mode='reflect', 
                                           power=2.0, 
                                           n_iter=32)

import soundfile as sf
sf.write("test1.wav", res, sashi_sr)